In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.feature_selection import VarianceThreshold

In [ ]:
df_bmw=pd.read_csv("data/bmw_pricing_v3.csv")

# Funciones

In [ ]:
def obtener_lista_variables(dataset):
    lista_numericas=[]
    lista_boolean=[]
    lista_categoricas=[]

    for i in dataset:
        if    (dataset[i].dtype.kind in ("f","i")) and len(dataset[i].unique())!= 2  and i not in target:
              lista_numericas.append(i)
        elif  (((dataset[i].dtype.kind in ("f","i")) and len(dataset[i].unique())== 2) or (set(df2[i].unique()) == {True ,False, np.nan} )or dataset[i].dtype.kind =="b")  and i not in target:
              lista_boolean.append(i)
        elif  (dataset[i].dtype.kind == "O")  and i not in target:
              lista_categoricas.append(i)
                
    return lista_numericas, lista_boolean, lista_categoricas

In [ ]:
def unicos(df):
    for i in df:
        if df[i].dtype.kind == "O":
             print(i,df[i].unique(),"\n")
        elif (df[i].dtype.kind=="f") or (df[i].dtype.kind=="i"):
            print(df.hist(i))     

In [ ]:
def valcounts(df):
    for i in df:
        if df[i].dtype.kind == "O":
            print(i,"\n", df[i].value_counts(), "\n")

In [ ]:
#columnas con nulos bajo umbral
def nulosbajosumbral(df,umbral):
    listanulos=[]
    for i in df:
        if (df[i].isnull().sum()/len(df))<umbral:
            listanulos.append(i)
    return listanulos        

In [ ]:
#remueve nulos de columnas especificas
def remuevenulos(df,columnas):
    for i in columnas:
        df.dropna(subset=[i], inplace=True)
    return df    

In [ ]:
# Rellena nulos con la moda de cada agrupacion de una columna
def modaxclase(lista,df,cclase):
    for i in lista:
        moda = df.groupby(cclase)[i].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else df[i].mode().iloc[0])
        df[i] = df[i].fillna(moda)
    return df

In [ ]:
def analisisvc(df,col):
    tabla=df[col].value_counts().reset_index()
    total=tabla["count"].sum()
    for i in range(0,len(tabla)):
        tabla.loc[i,"por"]=round(round(tabla.loc[i,"count"]/total,4)*100,2)
        if i==0:
            tabla.loc[i,"poracum"]=tabla.loc[i,"por"]
        else:
            tabla.loc[i,"poracum"]=tabla.loc[i-1,"poracum"]+tabla.loc[i,"por"]
    return tabla
            

In [ ]:
def obten_lista_eliminar(df, col, umbral):
  lista_borrar= []
  tabla=df[col].value_counts().reset_index()
  for i in range(0,len(tabla)):
    if (tabla.iloc[i,1]) < umbral:
      lista_borrar.append(tabla.iloc[i,0])
  return lista_borrar

In [ ]:
def eliminar_lista_df(dataset, columna, lista_eliminar):
  dataset.drop(dataset[dataset[columna].isin(lista_eliminar)].index, inplace=True)

In [ ]:
def obten_lista_otros(df, col, umbral):
    lista_otros= []
    tabla=df[col].value_counts().reset_index()
    for i in range(0,len(tabla)):
        if (tabla.iloc[i,1]) < umbral:
            lista_otros.append(tabla.iloc[i,0])
    return lista_otros

In [ ]:
def cambiar_a_otros(df, i):
    df.replace({i:"otros"}, inplace=True)

# Estructura de Datos

In [ ]:
df_bmw.info()

In [ ]:
df_bmw.shape

In [ ]:
df_bmw.head(5)

# Cleaning

In [ ]:
df2=df_bmw.copy()

In [ ]:
df2.info()

In [ ]:
df2[df2.duplicated(keep=False)] # sin duplicados

In [ ]:
for i in ["fecha_registro","fecha_venta"]:
    df2[i]=pd.to_datetime(df2[i])

In [ ]:
df2["precio"].isnull().sum()

In [ ]:
df2.dropna(subset=["precio"], inplace=True)

# EDA

In [ ]:
target= "precio"

In [ ]:
df2.describe().T

## Deteccion y tratamiento de nulos

In [ ]:
df2.isnull().sum()

In [ ]:
unicos(df2)

In [ ]:
del(df2["marca"]) # borrar marca, se esta analizando bmw

In [ ]:
del(df2["fecha_registro"]) # muchos nulos y se tiene el km para modelo de valorizacion por antiguedad/uso

In [ ]:
u=0.01
l1=nulosbajosumbral(df2,u)

In [ ]:
remuevenulos(df2,l1)
df2=df2.reset_index(drop=True)

In [ ]:
df2.isnull().sum()

In [ ]:
for i in df2:
    if set(df2[i].unique()) == {True ,False}:
        df2[i]=df2[i].astype(bool)

In [ ]:
valcounts(df2)

### Imputar Nulos

In [ ]:
df3=df2.copy()

In [ ]:
df3.info()

In [ ]:
df3.isnull().sum()

In [ ]:
for col in ["fecha_venta"]:
    df3[col + '_AÑO'] = df3[col].dt.year.astype(int)
    #df3[col + '_NOMBREDIA'] = df3[col].dt.day_name()
    ##O .dt.weekday_name
    df3[col + '_DIA'] = df3[col].dt.day.astype(int)
    df3[col + '_MES'] = df3[col].dt.month.astype(int)

In [ ]:
del(df3["fecha_venta"])

In [ ]:
lnum, lbool, lcat= obtener_lista_variables(df3)

In [ ]:
lnum

In [ ]:
lbool

In [ ]:
lcat

In [ ]:
modaxclase(lbool,df3,"modelo");

In [ ]:
for i in lbool:
    df3[i]=df3[i].astype(int)

In [ ]:
df3.isnull().sum()

In [ ]:
modaxclase(lcat,df3,"modelo");

In [ ]:
df3.isnull().sum()

## Analisis Univariable

In [ ]:
valcounts(df3)

In [ ]:
unicos(df3)

In [ ]:
df3["tipo_gasolina"]=df3["tipo_gasolina"].replace({"Diesel":"diesel"})

In [ ]:
df3.drop(df3.loc[(df3["tipo_gasolina"]=="hybrid_petrol") | (df3["tipo_gasolina"]=="electro"),"tipo_gasolina"].index,inplace=True)

In [ ]:
df3.reset_index(drop=True,inplace=True)

In [ ]:
del(df3["fecha_venta_DIA"])

In [ ]:
del(df3["fecha_venta_AÑO"])

In [ ]:
df3.info()

Tratamiento Outliers

In [ ]:
df4=df3.copy()

In [ ]:
sns.boxplot(x=df4["precio"])

In [ ]:
sns.boxplot(x=df4["potencia"])

In [ ]:
sns.boxplot(x=df4["km"])

In [ ]:
df4.iloc[df4.loc[(df4["km"]<=0) | (df4["km"]>500000),"km"].index]

In [ ]:
df4.drop(df4.loc[(df4["km"]<=0) | (df4["km"]>500000),"km"].index,inplace=True)


In [ ]:
df4=df4.reset_index(drop=True)

In [ ]:
df4.iloc[df4.loc[df4["potencia"]<=0,"potencia"].index]


In [ ]:
df4.drop(df4.loc[df4["potencia"]<=0,"potencia"].index,inplace=True)

In [ ]:
df4.reset_index(drop=True,inplace=True)

In [ ]:
df4.iloc[df4.loc[df4["precio"]>80000,"precio"].index]

In [ ]:
df4.drop(df4.loc[df4["precio"]>80000,"precio"].index,inplace=True)

In [ ]:
df4.reset_index(drop=True,inplace=True)

In [ ]:
df5=df4.copy()

In [ ]:
t=analisisvc(df5,"color")
t.head(20)
#modelo 150
#color 300
#tipo_coche 100

In [ ]:
otros=obten_lista_otros(df5,"modelo",150)

In [ ]:
for i in otros:
    cambiar_a_otros(df5,i)    

In [ ]:
otros=obten_lista_otros(df5,"tipo_coche",100)

In [ ]:
for i in otros:
    cambiar_a_otros(df5,i)   

# Preproccesing

In [ ]:
df6=df5.copy()

## Analisis de Correlación

In [ ]:
corr = df6.corr(numeric_only = True)

In [ ]:
corr.style.background_gradient(cmap='coolwarm')

## Analisis Variables vs Target

In [ ]:
for i in df6:
  if df5[i].dtype.kind in ("O"):
    plt.figure()
    sns.violinplot(x=i,y="precio", data=df6)    

In [ ]:
sns.pairplot(df6,x_vars=lbool, y_vars=target)
plt.show()

In [ ]:
df6.shape

## Reescalar Categoricas

In [ ]:
df6=pd.get_dummies(data=df6 ,columns=lcat)

## Analisis de Varianza

In [ ]:
vt = VarianceThreshold(threshold = 0.01)
vt.fit(df6)

cols_lowvar = df6.columns[vt.get_support()==False]
cols_lowvar

In [ ]:
df6=df6.drop(columns=['color_beige', 'color_green', 'color_orange', 'color_red'])

## Correlacion Final

In [ ]:
corr2 = df6.corr(numeric_only = True)
corr2.style.background_gradient(cmap='coolwarm')

In [88]:
df6.shape

(4800, 36)